In [1]:
# Importing some shizzle.

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import time
import requests
import datetime
import sys
import os
import pathlib
import seaborn as sns
from sklearn.metrics import roc_auc_score

location = 'rjsietsma'

read_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/'

In [2]:
# Define function to perform Shapiro-Wilk, Kolmogorov-Smirnov, Wilcoxon-/Mann-Whitney U -test, and a pearson correlation test.

def perform_stats(x, y):
    shapiro_x = stats.shapiro(x)[1]
    shapiro_y = stats.shapiro(y)[1]
    if shapiro_x >= 0.05:
        print("X is likely normally distributed.")
    else:
        print("X is likely not normally distributed.")
    if shapiro_y >= 0.05:
        print("Y is likely normally distributed.")
    else:
        print("Y is likely not normally distributed.")
    try:
        p_ks_xy = stats.ks_2samp(x, y)[1]
        print(f"The Kolmogorov-Smirnov test p-value: {p_ks_xy}")
    except Exception:
        print("Kolmogorov-Smirnov could not be performed!")
    try:
        p_wc_xy = stats.wilcoxon(x, y)[1]
        print(f"The Wilcoxon test p-value: {p_wc_xy}")
    except ValueError:
        p_mw_xy = stats.mannwhitneyu(x, y)[1]
        print(f"Wilcoxon could not be performed, \n"
             f"Using Mann-Whitney rank test p-value: {p_mw_xy}")
    except Exception:
        print("Neither Wilcoxon nor Mann-Whitney tests could be performed!")
    try:
        p_pears_xy = stats.pearsonr(x, y)
        print(f"The Pearson correlation: {p_pears_xy[0]},\n"
             f"p-value: {p_pears_xy[1]}")
    except Exception:
        print("Pearson correlation could not be performed!")

# Define function to calculate the Z-scores of given data.

def calc_z_scores(data):
    centered = data - data.mean(axis=0)
    return centered / centered.std(axis=0)

In [3]:
# Read in the data
full_snv_dataset = pd.read_csv(read_loc+'full_dataset.csv')
full_snv_dataset

,chr,pos,ref,alt,label,gene,consequence,capice,source
0,14,68196054,GCCCTG,G,LP/P,RDH12,FRAME_SHIFT,0.988552,train
1,20,10626717,TCA,T,LP/P,JAG1,STOP_GAINED,0.990329,train
2,20,10625898,CTG,C,LP/P,JAG1,FRAME_SHIFT,0.991696,train
3,20,10628741,AC,A,LP/P,JAG1,FRAME_SHIFT,0.983607,train
4,20,10625509,ACT,A,LP/P,JAG1,DOWNSTREAM,0.987999,train
...,...,...,...,...,...,...,...,...,...
456004,17,29556342,G,A,LP/P,NF1,SYNONYMOUS,0.000139,test
456005,11,5248177,A,T,LP/P,HBB,SYNONYMOUS,0.069934,test
456006,15,48787324,T,C,LP/P,FBN1,SYNONYMOUS,0.909190,test
456007,19,17947957,G,A,LP/P,JAK3,SYNONYMOUS,0.001238,test


In [7]:
full_mapped_ds = pd.read_csv(read_loc+'optimal_f1_full_ds_v2.csv', header=0)
full_mapped_ds

,gene,default_auc,default_f1,default_recall,default_fpr,default_spec,optimal_c,optimal_auc,optimal_f1,optimal_recall,optimal_fpr,optimal_spec,n_tot,n_benign,n_malign
0,RDH12,0.839161,0.833333,0.909091,0.090909,0.909091,0.020,0.839161,0.833333,0.909091,0.090909,0.909091,24,13,11
1,EIF2AK4,0.981481,0.909091,1.000000,0.000000,1.000000,0.020,0.981481,0.909091,1.000000,0.000000,1.000000,32,27,5
2,EIF2AK3,0.966667,0.842105,1.000000,0.000000,1.000000,0.020,0.966667,0.842105,1.000000,0.000000,1.000000,53,45,8
3,PRKAG2,0.981203,0.761905,1.000000,0.000000,1.000000,0.020,0.981203,0.761905,1.000000,0.000000,1.000000,282,266,16
4,JAG1,0.925388,0.928994,0.987421,0.012579,0.987421,0.025,0.937810,0.940120,0.987421,0.012579,0.987421,320,161,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2802,FLG2,1.000000,1.000000,1.000000,0.000000,1.000000,0.020,1.000000,1.000000,1.000000,0.000000,1.000000,4,3,1
2803,ACOX2,1.000000,1.000000,1.000000,0.000000,1.000000,0.020,1.000000,1.000000,1.000000,0.000000,1.000000,2,1,1
2804,SLITRK4,1.000000,1.000000,1.000000,0.000000,1.000000,0.020,1.000000,1.000000,1.000000,0.000000,1.000000,3,2,1
2805,POGLUT1,1.000000,1.000000,1.000000,0.000000,1.000000,0.020,1.000000,1.000000,1.000000,0.000000,1.000000,2,1,1


# Notes meeting 24-03-2020
- Get all genes that perform bad from the histograms earlier in this notebook
- Add FPR, recall, sensitivity and specificity to mapper function
- Add vkgl and test data to optimum AUC mapper
- *see school bookmark for gene panels* calculate auc and compare auc's
    - Immunodeficiency (big chunk)
    - Developmental delay
    - 5GPM everything except late onset, but is very large
    - More popular panels will have more information, possibly easier to interpret.
 - Phase 1:
     - Make a list of biases and other easy to test statistics per gene / chromosome
     - Grab genes that are difficult to predict and label them.
     - Apply (NaN)GMLVQ on (to predict) to show correlations / causations.


[google docs](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit) of what is interesting
